In [1]:
from keras import layers, models
from keras.datasets import mnist
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
mnist_data = r"mnist.npz"
(x_train, y_train), (x_test, y_test) = mnist.load_data(mnist_data)

In [3]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(16, 3, activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPool2D(padding='same'))
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPool2D(padding='same'))
model.add(layers.Conv2D(128, 1, activation='relu'))
model.add(layers.MaxPool2D(padding='same'))
model.add(layers.Conv2D(256, 1, activation='relu'))

#model.add(layers.MaxPool2D())
#model.add(layers.Flatten())  # This layer is not compatible with ARM-NN
#model.add(layers.Reshape([256])) # still implements as strided slice, not compatible
#model.add(layers.Lambda(lambda x: K.reshape(x, [-1, 256])))  # reshape from/to 4D tensor not supported
#model.add(layers.Dense(256, activation='relu'))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))


model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [5]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 128)        

In [6]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')


In [7]:
model.fit(x_train, y_train, epochs=7, batch_size=128)


Epoch 1/7
60000/60000 [==============================] - 6s 94us/step - loss: 0.4390 - accuracy: 0.8541
Epoch 2/7
60000/60000 [==============================] - 3s 45us/step - loss: 0.0973 - accuracy: 0.9706
Epoch 3/7
60000/60000 [==============================] - 3s 44us/step - loss: 0.0669 - accuracy: 0.9807
Epoch 4/7
60000/60000 [==============================] - 3s 46us/step - loss: 0.0537 - accuracy: 0.9842
Epoch 5/7
60000/60000 [==============================] - 3s 47us/step - loss: 0.0446 - accuracy: 0.9866
Epoch 6/7
60000/60000 [==============================] - 3s 45us/step - loss: 0.0385 - accuracy: 0.9884
Epoch 7/7
60000/60000 [==============================] - 3s 45us/step - loss: 0.0300 - accuracy: 0.9910


In [8]:
print(model.evaluate(x_test, y_test))

10000/10000 [==============================] - 1s 95us/step
[0.044228250057436524, 0.9861999750137329]


In [9]:
# Save model and weights to separated files.
with open("model.json", "w") as file:
    file.write(model.to_json())
model.save_weights("weights.h5")

# Save model and weights to the same file.
model.save('model.h5', include_optimizer=False)